
# Métodos Numéricos no Grao en Matemáticas

## Elementos de Fortran

### Doble precisión
- No se recomienda usar `0.d0`, por venir del F77, ni tampoco 0._8, por ser dependiente del compilador. En cambio:
```fortran
use iso_fortran_env, only :: real64
0._real64
```
 
- En la sección _Real literal constants_ del libro de Metcalf [1] se propone `0._real64`. Al final de esta sección, se indica que la doble precisión de una variable, por ejemplo `x`,  podría indicarse con la sentencia `double precision x` pero es obsoleta: _"we defer the description of this alternative but outmoded syntax to Appendix A.6"_.
- En el apéndice A.6, además de explicar que `double precision` es obsoleta, se presenta la sintaxis `d0`. Es decir, no se dice explícitamente que `d0` es  obsoleto pero sólo se menciona `d0` en un apéndice de material obsolescente.
- Hay otra buena razón para no usarlo. Los ficheros de datos escritos con `d0` pasan a ser exclusivo de Fortran, ya que otros lenguajes pueden no entender esta notación.

[1] Metcalf, Michael, and others, _Modern Fortran Explained: Incorporating Fortran 2023_, 6th edn (Oxford, 2023; online edn, Oxford Academic, 21 Mar. 2024), [https://doi.org/10.1093/oso/9780198876571.001.0001](https://doi.org/10.1093/oso/9780198876571.001.0001), accessed 17 Apr. 2025. 

### Arrays de doble precisión

In [1]:
cat dp.f90

program prueba
  use iso_fortran_env, only: real64
  implicit none
  integer :: i
  real(real64) :: x(5)
  print*, 'Crear un array de 5 posiciones con 1/3:'
  x = 1/3;                       print*, 'Opcion. 1 ', x
  x = 1/3.;                      print*, 'Opcion. 2 ', x
  x = 1_real64/3;                print*, 'Opcion. 3 ', x
  x = 1._real64/3;               print*, 'Opcion. 4 ', x
  print*, 'Crear un array de 5 posiciones con i/3, i = 1, 5:'
  x = [(real(i, real64), i = 1,5)]/3;  print*, 'Opcion. 5 ', x
  x = [real(real64):: (i, i = 1,5)]/3; print*, 'Opcion. 6 ', x
end program  


In [3]:
gfortran -o a.exe dp.f90
./a.exe

 Crear un array de 5 posiciones con 1/3:
 Opcion. 1    0.0000000000000000        0.0000000000000000        0.0000000000000000        0.0000000000000000        0.0000000000000000     
 Opcion. 2   0.33333334326744080       0.33333334326744080       0.33333334326744080       0.33333334326744080       0.33333334326744080     
 Opcion. 3    0.0000000000000000        0.0000000000000000        0.0000000000000000        0.0000000000000000        0.0000000000000000     
 Opcion. 4   0.33333333333333331       0.33333333333333331       0.33333333333333331       0.33333333333333331       0.33333333333333331     
 Crear un array de 5 posiciones con i/3, i = 1, 5:
 Opcion. 5   0.33333333333333331       0.66666666666666663        1.0000000000000000        1.3333333333333333        1.6666666666666667     
 Opcion. 6   0.33333333333333331       0.66666666666666663        1.0000000000000000        1.3333333333333333        1.6666666666666667     



### Operaciones vectoriales

Ejemplificaremos algunos elementos de Fortran con la factorización LU.

En este caso, el uso de operaciones vectoriales con secciones de matrices.

In [4]:
cat factor.f90

module factor
use iso_fortran_env, only: real64
implicit none

contains

subroutine lu (a, l, u)
real(real64), intent(in) :: a(:,:)
real(real64), intent(inout) :: l(:,:), u(:,:)
integer :: i, j

l = 0._real64
u = 0._real64
do j = 1, size(a,2)
  do i = 1, j
    u(i,j) = a(i,j) - dot_product(l(i,1:i-1), u(1:i-1,j))
  end do
  l(j,j) = 1._real64
  do i = j+1, size(a,2)
    l(i,j) = (a(i,j) - dot_product(l(i,1:j-1), u(1:j-1,j))) / u(j,j)
  end do
end do
end subroutine
end module


Se enfatiza el uso de:
- `intent`.
- `dot_product` en vez de un nuevo bucle `do`.

Como se verá, `factor()` es agnóstico respecto del uso de memoria estática o dinámica.

### Memoria estática

Primer ejemplo, uso de memoria estática:
- Muy sencillo y, en general, sólo recomendado para pruebas específicas.

In [5]:
cat main_static.f90 

program main_static
use iso_Fortran_env, only: real64
use factor, only: lu
implicit none
real(real64) :: a(3,3), l(3,3), u(3,3)
integer :: i

print *, 'Programa para testear la factorizacion LU, con la matriz estǭtica [9 2 1; 4 9 4; 7 8 9]:'
print*, ' '
a = reshape([real(real64):: 9,2,1, 4,9,4, 7,8,9], [3,3], order=[2,1])
call lu(a, l, u)
print*, 'Matriz L:'
do i=1,size(l,1)
  print*, l(i,:)
enddo  
print*, 'Matriz U:'
do i=1,size(u,1)
  print*, u(i,:)
enddo  
print*, 'Error ||L*U-A||_F:', norm2(matmul(l,u)-a)
end program


In [6]:
gfortran -o a.exe factor.f90 main_static.f90
./a.exe

 Programa para testear la factorizacion LU, con la matriz estática [9 2 1; 4 9 4; 7 8 9]:
  
 Matriz L:
   1.0000000000000000        0.0000000000000000        0.0000000000000000     
  0.44444444444444442        1.0000000000000000        0.0000000000000000     
  0.77777777777777779       0.79452054794520555        1.0000000000000000     
 Matriz U:
   9.0000000000000000        2.0000000000000000        1.0000000000000000     
   0.0000000000000000        8.1111111111111107        3.5555555555555554     
   0.0000000000000000        0.0000000000000000        5.3972602739726021     
 Error ||L*U-A||_F:   0.0000000000000000     


Sirve para ejemplificar el uso de:
- `reshape` con `order`.
- `use` con `only`.
- funciones intrínsecas como `matmul` y `norm2` que, para matrices, calcula la norma de Frobenius.

Podemos aprovechar para separar la compilación y el _linkado_, mostrar los `.o` y los `.mod`:

In [ ]:
# comando de compilado 
gfortran -c factor.f90 main_static.f90
# comando de linkado
gfortran -o a.exe factor.o main_static.o
ls *.o
ls *.mod

En PowerShell no se puede escribir un solo comando `ls *.o *.mod`, al contrario que en Cocalc.

### Memoria dinámica
- Alojamiento de memoria sencillo, sin comprobación de alojamiento previo o de errores.

In [8]:
cat main_dynamic.f90

program main_dynamic
use iso_Fortran_env, only: real64
use factor, only: lu
implicit none
real(real64), allocatable :: a(:,:), l(:,:), u(:,:)
integer :: n, i

print *, 'Programa para testear la factorizacion LU, con memoria dinamica.'
print*, ' '
print*, 'Dimension de la matriz:'
read*, n
print*, n
allocate(a(n,n), l(n,n), u(n,n))
print*, 'Valores de la matriz:'
do i = 1, n
  read*, a(i,:)
end do
call print_matrix('A', a)
call lu(a, l, u)
call print_matrix('L', l)
call print_matrix('U', u)
print*, 'Error ||L*U-A||_F:', norm2(matmul(l,u)-a)

contains

subroutine print_matrix(name, a)
character(*), intent(in) :: name
real(real64), intent(in) :: a(:,:)

print*, 'Matriz '//name//':'
do i=1,size(a,1)
  print*, a(i,:)
enddo
end subroutine
end program


In [ ]:
cat datos.txt

3 
9 2 1 
4 9 4 
7 8 9


In [23]:
gfortran -o a.exe factor.f90 main_dynamic.f90
cat datos.dat | ./a.exe

 Programa para testear la factorizacion LU, con memoria dinamica.
  
 Dimension de la matriz:
           3
 Valores de la matriz:
 Matriz A:
   9.0000000000000000        2.0000000000000000        1.0000000000000000     
   4.0000000000000000        9.0000000000000000        4.0000000000000000     
   7.0000000000000000        8.0000000000000000        9.0000000000000000     
 Matriz L:
   1.0000000000000000        0.0000000000000000        0.0000000000000000     
  0.44444444444444442        1.0000000000000000        0.0000000000000000     
  0.77777777777777779       0.79452054794520555        1.0000000000000000     
 Matriz U:
   9.0000000000000000        2.0000000000000000        1.0000000000000000     
   0.0000000000000000        8.1111111111111107        3.5555555555555554     
   0.0000000000000000        0.0000000000000000        5.3972602739726021     
 Error ||L*U-A||_F:   0.0000000000000000     


Sirve para ejemplificar el uso de:
- Lectura desde fichero con redireción `|` ó `<`.
- Uso de `contains` en el programa principal.

# Ejercicios
1. En `factor.f90`, incluir otras factorizaciones.